**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 2 - Project Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a project account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the project account;

6 - Queries the blockchain again to confirm the succesfull details setting.

Every account succesfully created is dumped to `datasets/projects.json` for later use and evidence collection.  

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"
    participant IPFS as "Interplanetary File System"

    Note over Platform, Blockchain: Deploy smart contract for account creation
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully
    
    Note over Platform, Blockchain: Project Account creation
    Platform->>Blockchain: Create Project Account
    Blockchain-->>Platform: Project Account Created Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm account creation
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
        
    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, IPFS: Set project details
    Platform->>IPFS: Send project JSON metadata to IPFS
    IPFS-->>Platform: Send metadata CID to the Platform
    Platform->>Blockchain: Set Project Metadata as Details in Blockchain
    Blockchain-->>Platform: Project Details Set Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm details setting
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
```

In [1]:
from name_generator import left, right
from utilities import *
from project_id_generator import * # - Random generated synthetic project ID
from iroha_helper import *
from loguru import logger
from dump_to_json import ProjectAccount, update_or_append_project_metadata
from ipfs_functions import *
from synthetic_project_json_ld import *

In [2]:
DOMAIN = "test"

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;  

In [3]:
# - Deploy contract
hash = create_account_contract()


2025-01-22 18:40:23.838 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_account_contract"
2025-01-22 18:40:23.844 | INFO     | iroha_helper:create_account_contract:41 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-22 18:40:23.845 | INFO     | iroha_helper:create_account_contract:41 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-22 18:40:24.498 | INFO     | iroha_helper:create_account_contract:41 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-22 18:40:24.527 | INFO     | iroha_helper:create_account_contract:41 - ('COMMITTED', 5, 0)
2025-01-22 18:40:24.529 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-22 18:40:24.549 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'68c43d557923010c94984acee01d3c440b62cc570af7a0a24170665a1a55342f'
2025-01-22 18:40:24.550 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: CAA81FECBA7826B57203

2 - Creates a project account into the blockchain;

In [4]:
project_private_key = IrohaCrypto.private_key()
project_public_key = IrohaCrypto.derive_public_key(project_private_key).decode("utf-8")

project_id = generate_project_id()

logger.info(f"Creating project with name: {project_id}")

# project_account = ProjectAccount(
#     project_id=f"{project_id}@{DOMAIN}",
#     public_key=project_public_key
# )

logger.info(f"Creating project with name: {project_id}")

# - Create account
address = integration_helpers.get_engine_receipts_address(hash)
create_project_account(address, project_id, DOMAIN, project_public_key)

# - Get account info
user_account_hash = get_account(address, project_id, DOMAIN)
integration_helpers.get_engine_receipts_result(user_account_hash)



2025-01-22 18:40:24.570 | INFO     | __main__:<module>:6 - Creating project with name: 98192
2025-01-22 18:40:24.574 | INFO     | __main__:<module>:13 - Creating project with name: 98192
2025-01-22 18:40:24.576 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-22 18:40:24.595 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'68c43d557923010c94984acee01d3c440b62cc570af7a0a24170665a1a55342f'
2025-01-22 18:40:24.596 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: CAA81FECBA7826B572035BA13F851557C3D92AB0
2025-01-22 18:40:24.597 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-22 18:40:24.599 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_project_account"
2025-01-22 18:40:24.620 | INFO     | iroha_helper:create_project_account:112 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-22 18:40:2

3 - Queries the blockchain to confirm the succesful creation of the account;

In [5]:
#Query - GetAccountDetail

details = get_account_detail(f"{project_id}@{DOMAIN}")

2025-01-22 18:40:30.670 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account_detail"
2025-01-22 18:40:30.692 | INFO     | iroha_helper:get_account_detail:279 - User Account id = 98192@test, {}
2025-01-22 18:40:30.693 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account_detail"


4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [6]:
# - Deploys the contract
hash = create_detail_contract()

2025-01-22 18:40:30.700 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_detail_contract"
2025-01-22 18:40:30.720 | INFO     | iroha_helper:create_detail_contract:56 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-22 18:40:30.723 | INFO     | iroha_helper:create_detail_contract:56 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-22 18:40:33.661 | INFO     | iroha_helper:create_detail_contract:56 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-22 18:40:33.692 | INFO     | iroha_helper:create_detail_contract:56 - ('COMMITTED', 5, 0)
2025-01-22 18:40:33.694 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-22 18:40:33.716 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'6371b09e741964e2de8dc14fc44c990ef0522f7240891fd54e0cc8c4435c5bb7'
2025-01-22 18:40:33.718 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 899304BE2832B3E43FD67D887

5 - Sets details for the project account:

In [7]:
# Generate synthetic project metadata JSON (assumed to be done already)
project_metadata_json = generate_synthetic_entry_ld()
project_metadata_cid = upload_json_to_ipfs(project_metadata_json)  # Assuming this is done before calling the function

# Call the function to update or append a new entry to the JSON-LD file
updated_entry_count = update_or_append_project_metadata(
    project_id, 
    project_metadata_json, 
    project_metadata_cid
)


address = integration_helpers.get_engine_receipts_address(hash)

# Setting account detail using the generated project metadata CID
hash = set_account_detail(
    address, 
    f"{project_id}@{DOMAIN}",  # Project ID as account
    "project_metadata_cid",    # The key we're setting
    project_metadata_cid       # The value (CID from IPFS)
)

2025-01-22 18:40:33.773 | INFO     | dump_to_json:update_or_append_project_metadata:216 - Updated existing project entry for project ID: 98192@test
2025-01-22 18:40:33.777 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'datasets/projects.json': 15
2025-01-22 18:40:33.779 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-22 18:40:33.798 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'6371b09e741964e2de8dc14fc44c990ef0522f7240891fd54e0cc8c4435c5bb7'
2025-01-22 18:40:33.799 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 899304BE2832B3E43FD67D88756F7F9F2505DD95
2025-01-22 18:40:33.800 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-22 18:40:33.802 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-01-22 18:40:33.816 | INFO     

6 - Queries the blockchain to confirm the succesful details setting

In [8]:
#Query - GetAccountDetail

details = get_account_detail(f"{project_id}@{DOMAIN}")

2025-01-22 18:40:36.785 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account_detail"
2025-01-22 18:40:36.809 | INFO     | iroha_helper:get_account_detail:279 - User Account id = 98192@test, { "admin@test" : { "project_metadata_cid" : "QmNiame5i7oGQ36VTufxBpQEshFsWUGszkXGtngXSfFtuk" } }
2025-01-22 18:40:36.811 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account_detail"


7 - Retrieves Project metadata (JSON-LD formatted) from IPFS

In [9]:
# Convert the JSON string to a Python dictionary
details_dict = json.loads(details)

# Now you can access the specific key like this
project_metadata_cid = details_dict["admin@test"]["project_metadata_cid"]

logger.info(f"Project Metadata CID: {project_metadata_cid}")

project_metadata = download_json_from_ipfs(project_metadata_cid)

logger.info(f"Project Metadata: {project_metadata}")

2025-01-22 18:40:36.824 | INFO     | __main__:<module>:7 - Project Metadata CID: QmNiame5i7oGQ36VTufxBpQEshFsWUGszkXGtngXSfFtuk
2025-01-22 18:40:36.833 | INFO     | __main__:<module>:11 - Project Metadata: {'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This research focuses on the benefits and challenges posed by cloud computing for smart transportation, with an emphasis on its potential for environmental restoration.', 'schema:endDate': '2027-07-10', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'Bill and Melinda Gates Foundation'}, 'schema:keywords': ['cloud computing', 'smart transportation', 'environmental restoration'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Svalbard, Norway'}, 'schema:name': 'Assessing the Benefits of cloud computing for smart transportation', 'schema:startDate': '2024-11-29'}
